In [4]:
import casadi as ca

In [ ]:
px, py, pz = ca.MX.sym('px', 1), ca.MX.sym('py', 1), ca.MX.sym('pz', 1)
vx, vy, vz = ca.MX.sym('vx', 1), ca.MX.sym('vy', 1), ca.MX.sym('vz', 1)
qw, qx, qy, qz = ca.MX.sym('qw',1), 
w = ca.MX.sym('w',3)   # angular velocity      wx wy wz

u1 = ca.MX.sym('u1')
u2 = ca.MX.sym('u2')
u3 = ca.MX.sym('u3')
u4 = ca.MX.sym('u4')

dpdt = ca.MX.sym('dpdt',3)     # dp_x/dt dp_y/dt dp_z/dt
dvdt = ca.MX.sym('dvdt',3)     # dv_x/dt dv_y/dt dv_z/dt
dqdt = ca.MX.sym('dqdt',4)     # dq_w/dt dq_x/dt dq_y/dt dq_z/dt
dwdt = ca.MX.sym('dwdt',3)     # dw_x/dt dw_y/dt dw_z/dt

x_dim = 13
u_dim = 4

x = ca.vertcat(p, v, q, w)
xdot = ca.vertcat(dpdt, dvdt, dqdt, dwdt)
u = ca.vertcat(u1, u2, u3, u4)

In [ ]:
def p_dot():
    return v

def v_dot():
    collective_thrust = self.u1 + self.u2 + self.u3 + self.u4
    force = cs.vertcat(0, 0, collective_thrust)
    acc_input = force/self.m
    g_vec = cs.vertcat(0.0, 0.0, -9.81)
    rotm = tools.quaternion2rotm(self.q)
    dvdt = cs.mtimes(rotm, acc_input) + g_vec
    return dvdt

In [ ]:
f_expl = ca.vertcat(
    v,
    
)